# Computational Work 01: Fundamentals of optimizations

###  The quasi-Newton approaches (BFGS, L-BFGS, and DFP) and Levenberg-Marquardt must be applied and compared to the following functions:
* Rosenbrock (2D)
* Rosenbrock (4D or 30D)
* Beale (2D)
* Booth (2D)
* Mathias (2D)
* Ackley (2D)
* Rastrigin (2D)
* Rastrigin (4D or 30D).
#### 1. Table with min, max, mean, and median objective functions values (30 runs with different initial solution for each run) must be presented and discussed.
#### 2.  Best results (solution, number of iterations to converge, number total of objective function evaluations, and objective function value) must be showed.
### Optional:
#### 3. In the 2D case studies, the isolines and convergence line of all optimizers must be presented.

In [1]:
import pandas as pd
from scipy import optimize as opt
import numpy as np
from IPython.display import display
import autograd.numpy as au
from autograd import grad, jacobian
from scipy.optimize import line_search

### DFP

In [2]:

def DFP(func, x0, args=(), jac=None, callback=None,
                   gtol=1e-5, norm=np.Inf, eps= np.sqrt(np.finfo(float).eps)):

    import matplotlib.pyplot as plt
    import numpy as np
    import autograd.numpy as au
    from autograd import grad, jacobian
    import scipy
    from scipy.optimize import line_search
    from scipy.optimize._optimize import _line_search_wolfe12
    
    Df=grad(func)
    #Df = lambda x:x

    x1 = [x0[0]]
    x2 = [x0[1]]
    Bf = np.eye(len(x0))
    nit =0
    nfev=0

    while True:
        nit+=1

        Grad = Df(x0)
        delta = -Bf.dot(Grad) 
        
        
        start_point = x0

        beta,fc, *_= _line_search_wolfe12(func, Df,start_point, delta,old_fval=func(x0),
                            gfk=Df(x0),old_old_fval=func(x0)+np.linalg.norm(Df(x0))/2)
        nfev+=fc

        if beta!=None:
            X = x0+ beta*delta
        if np.linalg.norm(Df(X)) < gtol:
            x1 += [X[0], ]
            x2 += [X[1], ]
            result = opt.OptimizeResult(
                fun=func(X),
                nfev=nfev,
                nit=nit,
                x=X,
                allvecs=(x1,x2)
            )
            return result
            #return X, func(X),(x1,x2)
        else:
            Dj = X - x0
            Gj = Df(X) - Grad 
            w1 = Dj 
            w2 = Bf.dot(Gj) 
            w1T = w1.T
            w2T = w2.T
            sigma1 = 1/(w1T.dot(Gj)) # 
            sigma2 = -1/(w2T.dot(Gj)) # 
            W1 = np.outer(w1, w1)
            W2 = np.outer(w2, w2)
            Delta = sigma1*W1 + sigma2*W2 # 
            Bf += Delta # 
            x0 = X # 
            x1 += [x0[0], ]
            x2 += [x0[1], ]
    

In [18]:
#Does : BFGS
#https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html
#results.append(opt.minimize(fun=func,x0=(np.random.uniform(-bound,bound),np.random.uniform(-bound,bound))

#Does L-BFGS
#https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html
#lbfgs = opt.minimize(fun=beale,x0=(np.random.uniform(-10,10),np.random.uniform(-10,10)),bounds=((-10,10),(-10,10)) ,method='L-BFGS-B')


#Does Levenberg-Maquardt
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html

#pd.set_option("display.precision",15)
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', '{:.2g}'.format)

def OptmizeResult_to_Series(results: list, nome: str):
    ''' Pega o resultado de opt.minimize. Retorna uma tupla:
        \n data: pd.Series com a min, máx, mean e median
        \n best: pd.Series com o melhor valor obtido
     '''

    #cria o dataframe de todos os resultados
    try:
        method_data=pd.DataFrame({
        "value": [ result.fun for result in results],
        "iterations": [ result.nit for result in results],
        "func_eval": [ result.nfev for result in results],
        "solutions": [ result.x for result in results]})
    except: #Esse except é necessário pq o Levenberg-Maquardt não dá o número de iterações
        method_data=pd.DataFrame({
        "value": [ result.fun for result in results],
        "iterations": '',
        "func_eval": [ result.nfev for result in results],
        "solutions": [ result.x for result in results]})

    method_data.sort_values(by="value",inplace=True)
    data=pd.Series({
        "nome":nome,
        "min":method_data["value"].min(),
        "max":method_data["value"].max(),
        "mean":method_data["value"].mean(),
        "median": method_data["value"].median(),
     })
    
    best=pd.Series(method_data.iloc[0])
    best["nome"]=nome
    best =best[["nome","value","iterations","func_eval","solutions"]]
    
    return data, best


def apply_methhods(func, search_space: float):
    '''Aplica os metodos nas funções objetiva. Metodos implementados: BFGS, LBFGS, Powell, Minimos Quadrados (Levenberg-Maquart)
        \n Retorna:
        \n *data*: pd.DataFrame com a min, máx, mean e median
        \n *best*: pd.DataFrame com o melhor valor obtido
        
    '''

    #cria as listas que vão armazenar os resutados
    bfgs =[]
    lbfgs=[]
    dfp =[]
    lq=[]

    #roda os métodos 30 vezes
    for _ in np.arange(30):
        try:
            #cria os pontos aleatórios de início
            x00=np.random.uniform(-search_space,search_space)
            x01=np.random.uniform(-search_space,search_space)
            
            #Roda os métodos e adiciona os resultados às listas
            bfgs.append(opt.minimize(fun=func,x0=(x00,x01), method='BFGS',options={'return_all':True}))
            lbfgs.append(opt.minimize(fun=func,x0=(x00,x01), bounds=((-search_space,search_space),(-search_space,search_space)) ,method='L-BFGS-B'))
            #dfp.append(DFP(func, x0=(x00,x01)))
            lq.append(opt.least_squares(func,x0=(x00,x01),bounds=(-search_space,search_space)))
        except: pass

    #Calcula os parâmetros pedidos e seleciona o melhor   
    bfgs_data,bfgs_best= OptmizeResult_to_Series(bfgs, 'bfgs')
    lbfgs_data, lbfgs_best= OptmizeResult_to_Series(lbfgs,'lbfgs')
    #powell_data, powell_best =OptmizeResult_to_Series(dfp,"dfp")
    lq_data,lq_best= OptmizeResult_to_Series(lq,"Minimos Quadrados")


    data = pd.DataFrame([bfgs_data,lbfgs_data,lq_data])#,powell_data])
    best= pd.DataFrame([bfgs_best,lbfgs_best,lq_best])#,powell_best])
    return data, best


### Rosenbrock 2D min=[1,1]

In [7]:
#rosenbrock = lambda x: (1-x[0])**2+ 100 * (x[1] - x[0]**2)**2
rosenbrock = lambda x:sum (( (1-x[i])**2+ 100 * (x[i+1] - x[i]**2)**2  for i in range(len(x)-1)))

search_space = 10

data, best = apply_methhods(rosenbrock,search_space=search_space)
display(data)
display(best)


,nome,min,max,mean,median
0,bfgs,1.6e-13,3.7e-11,1.6e-11,1.8e-11
1,lbfgs,7.4e-13,2.5e-11,9.5e-12,9.1e-12
2,Minimos Quadrados,[0.047996893676955074],[15.977182825132324],[4.33323568098781],0.31
3,dfp,1.7e-17,1e-10,8.7e-12,3e-14


,nome,value,iterations,func_eval,solutions
1,bfgs,1.6e-13,78,303,"[0.9999995961757615, 0.9999991910391266]"
8,lbfgs,7.4e-13,28,111,"[0.9999993756654665, 0.9999986918158139]"
12,Minimos Quadrados,[0.047996893676955074],,200,"[0.7815108213583795, 0.6091486593412205]"
6,dfp,1.7e-17,1003,1073,"[1.0000000041382404, 1.0000000082438951]"


### Rosenbrock 30D

In [25]:
search_space = 10

bfgs =[]
lbfgs=[]
powell =[]
lq=[]
N=30
for _ in np.arange(30):
        x0=[np.random.uniform(-search_space,search_space) for _ in range((N))]
        bounds=[(-search_space,search_space) for _ in range(N)]
        bfgs.append(opt.minimize(fun=rosenbrock,x0=x0, method='BFGS'))
        lbfgs.append(opt.minimize(fun=rosenbrock,x0=x0 , bounds=bounds ,method='L-BFGS-B'))
        #powell.append(opt.minimize(rosenbrock, x0=x0  ,method="Powell",bounds=bounds))
        lq.append(opt.least_squares(rosenbrock,x0=x0 ,bounds=(-search_space,search_space)))

        
bfgs_data,bfgs_best= OptmizeResult_to_Series(bfgs, 'bfgs')
lbfgs_data, lbfgs_best= OptmizeResult_to_Series(lbfgs,'lbfgs')
#powell_data, powell_best = OptmizeResult_to_Series(powell,"powell")
lq_data,lq_best= OptmizeResult_to_Series(lq,"Minimos Quadrados")


data = pd.DataFrame([bfgs_data,lbfgs_data,lq_data])
best= pd.DataFrame([bfgs_best,lbfgs_best,lq_best])

display(data)
display(best)
#10D = 1 min
#30D= 6 min

,nome,min,max,mean,median
0,bfgs,4.9e-11,4,0.4,6.3e-11
1,lbfgs,2e-10,4,1.2,5.3e-10
2,Minimos Quadrados,[0.20066570665116268],[10.505995794499112],[6.10953898888032],6.2


,nome,value,iterations,func_eval,solutions
24,bfgs,4.9e-11,415,14334,"[1.0000000754767862, 1.0000000727205276, 1.000..."
10,lbfgs,2e-10,103,3627,"[1.000000020931495, 1.0000000033644052, 0.9999..."
10,Minimos Quadrados,[0.20066570665116268],,3000,"[0.9999999789307648, 0.9999999726947992, 0.999..."


### Beale (min= [3,0.5])

In [6]:
beale= lambda x: (1.5-x[0]+x[0]*x[1])**2 + (2.25 - x[0] + x[0]*(x[1]**2))**2 + (2.625 -x[0] +x[0]*(x[1]**3) )**2
search_space = 4.5

data, best = apply_methhods(beale,search_space=search_space)
display(data)
display(best)

,nome,min,max,mean,median
0,bfgs,5.9e-15,7.4,0.41,3.1e-12
1,lbfgs,6.1e-15,0.76,0.13,2.7e-13
2,Minimos Quadrados,[8.206567020267974e-05],[0.042097190524243294],[0.007366505370384327],0.0011
3,dfp,3.1e-16,2.9e-11,4.7e-12,2.5e-13


,nome,value,iterations,func_eval,solutions
21,bfgs,5.9e-15,30,126,"[2.999999916557691, 0.49999999364531933]"
11,lbfgs,6.1e-15,14,60,"[2.9999999248682854, 0.49999999645012344]"
10,Minimos Quadrados,[8.206567020267974e-05],,200,"[3.022838570283971, 0.5054434381219882]"
4,dfp,3.1e-16,13,16,"[2.999999958412891, 0.49999999093001646]"


## Booth min =[1,3]

In [4]:
booth = lambda x: np.square(x[0]+2*x[1]-7) + np.square(2*x[0]+x[1]-5)

search_space = 10

data, best = apply_methhods(booth,search_space=search_space)
display(data)
display(best)



,nome,min,max,mean,median
0,bfgs,2e-16,2.1e-11,1.2e-12,2.4e-14
1,lbfgs,1.4e-16,7.4e-12,1e-12,1.8e-13
2,Minimos Quadrados,[5.368990649996446e-08],[6.446836850886881e-07],[3.5576175629840523e-07],3.6e-07
3,dfp,0,2.6e-12,1.6e-13,2.3e-16


,nome,value,iterations,func_eval,solutions
29,bfgs,2e-16,6,21,"[0.9999999991151898, 2.999999994428932]"
20,lbfgs,1.4e-16,4,18,"[0.9999999972219071, 2.9999999972225746]"
15,Minimos Quadrados,[5.368990649996446e-08],,28,"[0.9998367560249718, 3.00009676491307]"
20,dfp,0,11,11,"[0.9999999999999999, 3.0]"


### Matyas min=[0,0]

In [5]:
matyas= lambda x: 0.26*(x[0]**2+x[1]**2) -0.48*x[0]*x[1]

search_space = 10

data, best = apply_methhods(matyas,search_space=search_space)
display(data)
display(best)

,nome,min,max,mean,median
0,bfgs,3.2e-16,3.7e-10,4.7e-11,4.5e-13
1,lbfgs,1.7e-16,4e-11,6.7e-12,2.2e-12
2,Minimos Quadrados,[1.0405950111580577e-06],[1.7434059198854358e-06],[1.4688784264014092e-06],1.5e-06
3,dfp,5e-18,1.3e-10,1.8e-11,2.9e-12


,nome,value,iterations,func_eval,solutions
26,bfgs,3.2e-16,8,30,"[-8.876980230982797e-08, -8.89467104142215e-08]"
4,lbfgs,1.7e-16,3,15,"[-6.441133653368688e-08, -6.441425119515289e-08]"
22,Minimos Quadrados,[1.0405950111580577e-06],,7,"[0.005183414959881284, 0.00495131384258631]"
16,dfp,5e-18,5,6,"[-1.1371997704291345e-08, -1.1026430088915517e..."


### Rastrigin 2D min=[0,0]

In [19]:
rastrigin= lambda X: 10 + sum([(x**2 - 10 * np.cos(2 * np.pi * x)) for x in X])
search_space = 5.12

data, best = apply_methhods(rastrigin,search_space=search_space)
display(data)
display(best)


,nome,min,max,mean,median
0,bfgs,-9,31,4.5,2.9
1,lbfgs,-9,31,2.4,-0.05
2,Minimos Quadrados,[-3.0015874941113907e-07],[30.792967657930635],[5.553217478172522],1.9e-07


,nome,value,iterations,func_eval,solutions
0,bfgs,-9,6,30,"[-1.135032627944694e-08, 0.9949586286417684]"
0,lbfgs,-9,5,27,"[4.510787518041795e-09, 0.9949586315772456]"
23,Minimos Quadrados,[-3.0015874941113907e-07],,32,"[0.8467336198972638, -2.061733672459886]"


### Rastrigin 30D min=[0,0]

In [24]:
search_space = 5.12

bfgs =[]
lbfgs=[]
powell =[]
lq=[]
N=30
for _ in np.arange(30):
        x0=[np.random.uniform(-search_space,search_space) for _ in range((N))]
        bounds=[(-search_space,search_space) for _ in range(N)]
        
        bfgs.append(opt.minimize(fun=rastrigin,x0=x0, method='BFGS'))
        lbfgs.append(opt.minimize(fun=rastrigin,x0=x0 , bounds=bounds ,method='L-BFGS-B'))
        #powell.append(opt.minimize(rastrigin, x0=x0  ,method="Powell",bounds=bounds))
        lq.append(opt.least_squares(rastrigin,x0=x0 ,bounds=(-search_space,search_space)))

        
bfgs_data,bfgs_best= OptmizeResult_to_Series(bfgs, 'bfgs')
lbfgs_data, lbfgs_best= OptmizeResult_to_Series(lbfgs,'lbfgs')
#powell_data, powell_best = OptmizeResult_to_Series(powell,"powell")
lq_data,lq_best= OptmizeResult_to_Series(lq,"Minimos Quadrados")


data = pd.DataFrame([bfgs_data,lbfgs_data,lq_data])
best= pd.DataFrame([bfgs_best,lbfgs_best,lq_best])

display(data)
display(best)
#10D = 1 min
#30D= 6 min

,nome,min,max,mean,median
0,bfgs,-1e+02,40,-25,-32
1,lbfgs,-1.6e+02,16,-73,-76
2,Minimos Quadrados,[-1.7270023242588195e-06],[40.324145866272275],[6.75575767370701],-1.8e-15


,nome,value,iterations,func_eval,solutions
27,bfgs,-1e+02,54,3286,"[-2.92091076660333e-08, -0.9949586436533235, -..."
27,lbfgs,-1.6e+02,13,558,"[0.9949586384907562, -7.79321799503215e-09, -2..."
2,Minimos Quadrados,[-1.7270023242588195e-06],,33,"[4.583316239913953, -1.9423612176931273, 0.944..."


### Ackley min = (0,0)

In [20]:
ackley= lambda x:   (-20*np.exp(-0.2*np.sqrt(0.5*(x[0]**2 + x[1]**2)))- 
                    np.exp(0.5*np.cos(2*np.pi*x[0])+np.cos(2*np.pi * x[1])) +np.e +20)
search_space = 32

data, best = apply_methhods(ackley,search_space=search_space)
display(data)
display(best)


,nome,min,max,mean,median
0,bfgs,7.1,18,16,17
1,lbfgs,3.6,18,15,17
2,Minimos Quadrados,[1.4797478797845542e-09],[18.145124955749832],[13.882669298297134],17


,nome,value,iterations,func_eval,solutions
15,bfgs,7.1,9,81,"[0.9956669703861738, 3.9913118968858243]"
14,lbfgs,3.6,10,84,"[0.9895595410240363, 1.9895047148649492]"
15,Minimos Quadrados,[1.4797478797845542e-09],,47,"[0.12256817049916327, 0.1039690800449424]"
